In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EuroSAT_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(EuroSAT_CNN, self).__init__()
        
        # Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.drop1 = nn.Dropout(0.25)
        
        # Block 4
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.drop2 = nn.Dropout(0.25)
        
        # Block 5
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.drop3 = nn.Dropout(0.25)
        
        # Classifier
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512, 256)
        self.drop4 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(256, num_classes)
        
        self.leaky_relu = nn.LeakyReLU(0.1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.leaky_relu(x)
        x = self.pool(x)
        
        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.leaky_relu(x)
        x = self.pool(x)
        
        # Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.leaky_relu(x)
        x = self.pool(x)
        x = self.drop1(x)
        
        # Block 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.leaky_relu(x)
        x = self.pool(x)
        x = self.drop2(x)
        
        # Block 5
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.leaky_relu(x)
        x = self.pool(x)
        x = self.drop3(x)
        
        # Head
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.leaky_relu(x)
        x = self.drop4(x)
        
        x = self.fc2(x)
        return x

In [14]:
import torch
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def convert_weights(h5_path, pth_output_path):
    print(f"Loading Keras model from {h5_path}...")
    # Load Keras model
    keras_model = load_model(h5_path, compile=False)
    
    # Instantiate PyTorch model
    # Note: Keras model had 10 classes (EuroSAT default)
    torch_model = EuroSAT_CNN(num_classes=10) 
    torch_model.eval()
    
    # Helper to get layer weights from Keras model
    # We iterate manually to match the specific architecture
    k_layers = [l for l in keras_model.layers if len(l.get_weights()) > 0]
    
    # Mapping logic:
    # Keras Conv2D weights: [H, W, In, Out] -> PyTorch: [Out, In, H, W]
    # Keras Dense weights: [In, Out] -> PyTorch: [Out, In]
    
    # We define the sequence of PyTorch layers that have weights
    # based on the order they appear in the forward pass / __init__
    torch_layer_names = [
        'conv1', 'bn1', 
        'conv2', 'bn2', 
        'conv3', 'bn3', 
        'conv4', 'bn4', 
        'conv5', 'bn5', 
        'fc1', 'fc2'
    ]
    
    k_idx = 0
    state_dict = torch_model.state_dict()
    
    print("Transferring weights...")
    
    for name in torch_layer_names:
        layer = getattr(torch_model, name)
        
        # Skip Keras layers that don't have weights (Dropout, Pooling, etc are skipped in k_layers list)
        while k_idx < len(k_layers) and len(k_layers[k_idx].get_weights()) == 0:
            k_idx += 1
            
        if k_idx >= len(k_layers):
            break
            
        k_layer = k_layers[k_idx]
        k_weights = k_layer.get_weights()
        
        print(f"Processing {name} <- Keras: {k_layer.name}")
        
        if isinstance(layer, torch.nn.Conv2d):
            # Keras: [H, W, In, Out], Bias [Out]
            # PyTorch: [Out, In, H, W], Bias [Out]
            weight = k_weights[0]
            bias = k_weights[1] if len(k_weights) > 1 else None
            
            # Transpose: (H, W, In, Out) -> (Out, In, H, W)
            weight = np.transpose(weight, (3, 2, 0, 1))
            
            state_dict[f'{name}.weight'] = torch.from_numpy(weight)
            if bias is not None:
                state_dict[f'{name}.bias'] = torch.from_numpy(bias)
                
        elif isinstance(layer, torch.nn.BatchNorm2d):
            # Keras BN: [gamma, beta, mean, variance]
            # PyTorch BN: weight(gamma), bias(beta), running_mean, running_var
            gamma, beta, mean, var = k_weights
            
            state_dict[f'{name}.weight'] = torch.from_numpy(gamma)
            state_dict[f'{name}.bias'] = torch.from_numpy(beta)
            state_dict[f'{name}.running_mean'] = torch.from_numpy(mean)
            state_dict[f'{name}.running_var'] = torch.from_numpy(var)
            
        elif isinstance(layer, torch.nn.Linear):
            # Keras: [In, Out]
            # PyTorch: [Out, In]
            weight = k_weights[0]
            bias = k_weights[1] if len(k_weights) > 1 else None
            
            weight = np.transpose(weight, (1, 0))
            
            state_dict[f'{name}.weight'] = torch.from_numpy(weight)
            if bias is not None:
                state_dict[f'{name}.bias'] = torch.from_numpy(bias)
        
        k_idx += 1

    # Save
    torch_model.load_state_dict(state_dict)
    torch.save(torch_model.state_dict(), pth_output_path)
    print(f"Successfully saved converted model to {pth_output_path}")

# Run the conversion
h5_file = "final.h5" 
convert_weights(h5_file, "eurosat_converted.pth")

Loading Keras model from final.h5...


c:\Users\moihammed\miniconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Transferring weights...
Processing conv1 <- Keras: conv2d_10
Processing bn1 <- Keras: batch_normalization_10
Processing conv2 <- Keras: conv2d_11
Processing bn2 <- Keras: batch_normalization_11
Processing conv3 <- Keras: conv2d_12
Processing bn3 <- Keras: batch_normalization_12
Processing conv4 <- Keras: conv2d_13
Processing bn4 <- Keras: batch_normalization_13
Processing conv5 <- Keras: conv2d_14
Processing bn5 <- Keras: batch_normalization_14
Processing fc1 <- Keras: dense_4
Processing fc2 <- Keras: dense_5
Successfully saved converted model to eurosat_converted.pth


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

def fine_tune():
    # --- Configuration ---
    DATA_DIR = 'Egypt_Data_Split' # Assumes folders 'train' and 'test' inside
    CONVERTED_MODEL_PATH = 'eurosat_converted.pth'
    BATCH_SIZE = 32
    LEARNING_RATE = 0.0001
    EPOCHS = 10
    IMG_SIZE = 128
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {DEVICE}")

    # --- Data Transforms ---
    # Matches the preprocessing in the notebook (rescale 1/255 is handled by ToTensor)
    train_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(), # Converts 0-255 to 0.0-1.0
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])

    # --- Load Data ---
    train_dir = os.path.join(DATA_DIR, 'train')
    test_dir = os.path.join(DATA_DIR, 'test')

    if not os.path.exists(train_dir):
        print(f"Error: Directory {train_dir} not found.")
        return

    train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
    val_dataset = datasets.ImageFolder(test_dir, transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    num_new_classes = len(train_dataset.classes)
    print(f"Detected {num_new_classes} new Egypt classes: {train_dataset.classes}")

    # --- Load Model ---
    # 1. Initialize with original 10 classes to load weights correctly
    model = EuroSAT_CNN(num_classes=10)
    
    # 2. Load the converted weights
    try:
        model.load_state_dict(torch.load(CONVERTED_MODEL_PATH))
        print("Converted weights loaded successfully.")
    except FileNotFoundError:
        print("Please run the conversion script first!")
        return

    # 3. EXPAND the final layer to support 10 original + 3 new classes = 13 total
    total_classes = 10 + num_new_classes  # 10 EuroSAT + 3 Egypt = 13 classes
    in_features = model.fc2.in_features
    
    # Save old weights
    old_fc2_weight = model.fc2.weight.data.clone()
    old_fc2_bias = model.fc2.bias.data.clone()
    
    # Create new larger final layer
    model.fc2 = nn.Linear(in_features, total_classes)
    
    # Copy old weights to the new layer (for first 10 classes)
    model.fc2.weight.data[:10] = old_fc2_weight
    model.fc2.bias.data[:10] = old_fc2_bias
    
    # Initialize new class weights randomly (for classes 10-12)
    nn.init.xavier_uniform_(model.fc2.weight.data[10:])
    nn.init.zeros_(model.fc2.bias.data[10:])
    
    print(f"Model expanded to predict {total_classes} classes:")
    print(f"  - Classes 0-9: Original EuroSAT classes (weights preserved)")
    print(f"  - Classes 10-{total_classes-1}: New Egypt classes (Crop Data, Desert Data, Urban Data)")
    
    model = model.to(DEVICE)

    # --- Setup Training ---
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # --- Adjust labels: Egypt classes should be mapped to indices 10, 11, 12 ---
    # The DataLoader gives labels 0, 1, 2 for Egypt classes
    # We need to shift them to 10, 11, 12
    
    # --- Training Loop ---
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(DEVICE)
            # Shift labels from [0,1,2] to [10,11,12]
            labels = labels + 10
            labels = labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {running_loss/len(train_loader):.4f} | Train Acc: {epoch_acc:.2f}%")

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(DEVICE)
                # Shift labels from [0,1,2] to [10,11,12]
                labels = labels + 10
                labels = labels.to(DEVICE)
                
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        print(f"Validation Acc: {val_acc:.2f}%")

    # Save Fine-tuned model
    torch.save(model.state_dict(), 'finetuned_egypt_model.pth')
    print(f"\nFine-tuning complete. Model saved with {total_classes} classes.")
    print("Class mapping:")
    print("  0-9: Original EuroSAT classes")
    print("  10: Crop Data")
    print("  11: Desert Data")
    print("  12: Urban Data")

if __name__ == "__main__":
    fine_tune()

Using device: cuda
Detected 3 classes: ['Crop Data', 'Desert Data', 'Urban Data']
Converted weights loaded successfully.
Epoch [1/10] Loss: 0.7472 | Train Acc: 69.36%
Epoch [1/10] Loss: 0.7472 | Train Acc: 69.36%
Validation Acc: 88.57%
Validation Acc: 88.57%
Epoch [2/10] Loss: 0.3680 | Train Acc: 90.66%
Epoch [2/10] Loss: 0.3680 | Train Acc: 90.66%
Validation Acc: 94.64%
Validation Acc: 94.64%
Epoch [3/10] Loss: 0.2056 | Train Acc: 95.06%
Epoch [3/10] Loss: 0.2056 | Train Acc: 95.06%
Validation Acc: 96.07%
Validation Acc: 96.07%
Epoch [4/10] Loss: 0.1383 | Train Acc: 96.32%
Epoch [4/10] Loss: 0.1383 | Train Acc: 96.32%
Validation Acc: 96.07%
Validation Acc: 96.07%
Epoch [5/10] Loss: 0.1038 | Train Acc: 97.21%
Epoch [5/10] Loss: 0.1038 | Train Acc: 97.21%
Validation Acc: 97.50%
Validation Acc: 97.50%
Epoch [6/10] Loss: 0.0917 | Train Acc: 97.04%
Epoch [6/10] Loss: 0.0917 | Train Acc: 97.04%
Validation Acc: 97.50%
Validation Acc: 97.50%
Epoch [7/10] Loss: 0.0930 | Train Acc: 97.04%
Epoch

In [ ]:
import torch
from torchsummary import torchsummary
from torchvision import datasets

# Load the fine-tuned model with 13 classes
model = EuroSAT_CNN(num_classes=13)  # 10 EuroSAT + 3 Egypt = 13 classes
model.load_state_dict(torch.load('finetuned_egypt_model.pth'))
model.eval()

print("="*60)
print("MODEL ARCHITECTURE")
print("="*60)
print(model)
print("\n")

# Define all class names
eurosat_classes = [
    "AnnualCrop", "Forest", "HerbaceousVegetation", "Highway", "Industrial",
    "Pasture", "PermanentCrop", "Residential", "River", "SeaLake"
]

egypt_classes = ["Crop Data", "Desert Data", "Urban Data"]

all_classes = eurosat_classes + egypt_classes

print("="*60)
print("ALL CLASSES THE MODEL CAN PREDICT")
print("="*60)
for idx, class_name in enumerate(all_classes):
    if idx < 10:
        print(f"Class {idx}: {class_name} (EuroSAT - original)")
    else:
        print(f"Class {idx}: {class_name} (Egypt - new)")

print("\n")
print("="*60)
print("MODEL SUMMARY")
print("="*60)
try:
    from torchsummary import summary
    summary(model, (3, 128, 128))
except:
    print("Install torchsummary for detailed model summary: pip install torchsummary")
    print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

MODEL ARCHITECTURE
EuroSAT_CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop1): Dropout(p=0.25, inplace=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop2): Dropout(p=0.25, inplace=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop3): Dropout(p=0.25, inplace=False)
  (gl

In [ ]:
import torch

# Load the fine-tuned model
model = EuroSAT_CNN(num_classes=13)
model.load_state_dict(torch.load('finetuned_egypt_model.pth'))
model.eval()

# Get number of classes from the model
num_classes = model.fc2.out_features

print("="*60)
print("MODEL INFORMATION")
print("="*60)
print(f"Total number of classes: {num_classes}")
print(f"  - Original EuroSAT classes: 10 (indices 0-9)")
print(f"  - New Egypt classes: 3 (indices 10-12)")
print("="*60)

# Define all class names
all_classes = [
    "0: AnnualCrop (EuroSAT)",
    "1: Forest (EuroSAT)",
    "2: HerbaceousVegetation (EuroSAT)",
    "3: Highway (EuroSAT)",
    "4: Industrial (EuroSAT)",
    "5: Pasture (EuroSAT)",
    "6: PermanentCrop (EuroSAT)",
    "7: Residential (EuroSAT)",
    "8: River (EuroSAT)",
    "9: SeaLake (EuroSAT)",
    "10: Crop Data (Egypt)",
    "11: Desert Data (Egypt)",
    "12: Urban Data (Egypt)"
]

print("\nALL CLASSES:")
for class_name in all_classes:
    print(f"  {class_name}")

3